In [1]:
# %%capture
# %pip install -U bitsandbytes
# %pip install -U transformers
# %pip install -U peft
# %pip install -U accelerate
# %pip install -U datasets
# %pip install -U trl

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer

In [3]:
base_model = 'microsoft/phi-2'
new_model = 'models/phi-2-finetuned'
output_dir = 'results/phi-2-finetuned'

# Load your dataset
dataset_name = "tatsu-lab/alpaca"  # Replace with your dataset
dataset = load_dataset(dataset_name)

# Select just 5 enteries 
# dataset = dataset['train'].shuffle(seed=42).select(range(5))

# Split the dataset into train and test with a fixed seed
train_test_split = dataset['train'].train_test_split(test_size=0.0005, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
# Load base model(Phi-2)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)
model.get_memory_footprint()

In [6]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    logging_steps=100,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    max_seq_length= 2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [8]:
trainer.train()

/work/nlx505/finetuning_llms/finetune/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,0.959800
200,0.772300
300,0.750000
400,0.725900
500,0.763800
600,0.700900
700,0.758600
800,0.762200
900,0.694300
1000,0.726300


TrainOutput(global_step=25988, training_loss=0.691098371872411, metrics={'train_runtime': 14893.1423, 'train_samples_per_second': 3.49, 'train_steps_per_second': 1.745, 'total_flos': 9.36912306859008e+16, 'train_loss': 0.691098371872411, 'epoch': 1.0})

In [9]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('models/phi-2-finetuned/tokenizer_config.json',
 'models/phi-2-finetuned/special_tokens_map.json',
 'models/phi-2-finetuned/vocab.json',
 'models/phi-2-finetuned/merges.txt',
 'models/phi-2-finetuned/added_tokens.json',
 'models/phi-2-finetuned/tokenizer.json')

**Evaluate finetuned model**

In [ ]:
from transformers import logging
logging.set_verbosity_error()

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(new_model)
tokenizer = AutoTokenizer.from_pretrained(new_model)

In [ ]:
prompt = test_dataset[0]['text']
pipe = pipeline(task="text-generation", model=new_model, tokenizer=new_model)
result = pipe(prompt, max_new_tokens=50)
print(result[0]['generated_text'].split('Response:\n')[1])

In [ ]:
print(test_dataset[0]['output'])